In [2]:
import numpy as np
import cv2

In [3]:
img1 = cv2.imread("image1.webp")
img2 = cv2.imread("image2.webp")

In [5]:
color = (255, 0, 0)

start1 = (175, 450)
end1 = (275, 600)

start2 = (125, 450)
end2 = (225, 600)

In [6]:
cv2.rectangle(img1, start1, end1, color, 2);
cv2.rectangle(img2, start2, end2, color, 2);

In [111]:
rect1 = img1[start1[1] : end1[1], start1[0] : end1[0]]
rect2 = img2[start2[1] : end2[1], start2[0] : end2[0]]

rect1 = cv2.GaussianBlur(rect1, (5, 5), 0)
rect2 = cv2.GaussianBlur(rect2, (5, 5), 0)

In [112]:
sift = cv2.SIFT_create(contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)

In [113]:
# Find Keypoints
keypoints1, descriptors1 = sift.detectAndCompute(rect1, None)
keypoints2, descriptors2 = sift.detectAndCompute(rect2, None)

bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False) # Brute force matcher
matches = bf.knnMatch(descriptors1, descriptors2, k = 2) # Using knnMatch toi find k (=2) closest points 
# matches = sorted(matches, key=lambda x: x.distance)

In [114]:
# Implementing Lowe's ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance: # Take only if first ine is better than the second by 0.75 times
        good_matches.append(m)

In [119]:
# Extract points from good matches
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Compute the homography matrix using RANSAC
H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 100.0)

# Mask is a list of inlier points (1) and outlier points (0)
matches_mask = mask.ravel().tolist()

In [120]:
matched_image = cv2.drawMatches(rect1, keypoints1, rect2, keypoints2, good_matches, None, singlePointColor=None, matchesMask = matches_mask, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2.imshow("Final image", matched_image)
cv2.waitKey(0)
cv2.destroyAllWindows()